In [1]:
import gc
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau

from HW_base import evaluate_build, focus_build, plt_scatter
from HW_keras import hw_layer

In [2]:
fname = 'Keras_F5-E128-F60_Linear-LSTM128x4-FC-SELU-FC_Small-batch'

In [3]:
data_train_df = pd.read_csv('./Database/train.csv')
data_test_df = pd.read_csv('./Database/test.csv')

x_columns = ['R', 'C', 'time_step', 'u_in', 'u_out']
y_columns = ['pressure']

data_train = data_train_df[x_columns].values.astype(np.float32)
data_train = data_train.reshape(-1, 80, data_train.shape[-1])

target_train = data_train_df[y_columns].values.astype(np.float32)
target_train = target_train.reshape(-1, 80, target_train.shape[-1])

data_test = data_test_df[x_columns].values.astype(np.float32)
data_test = data_test.reshape(-1, 80, data_test.shape[-1])

del data_train_df
del data_test_df
gc.collect()

0

In [4]:
np.random.seed(121212)
data_idx = np.arange(len(data_train))
np.random.shuffle(data_idx)

train_index = data_idx[:int(len(data_idx)*0.8)]
valid_index = data_idx[int(len(data_idx)*0.8):]

x_train, y_train = data_train[train_index], target_train[train_index]
x_valid, y_valid = data_train[valid_index], target_train[valid_index]
x_test = data_test

del data_train
del data_test
gc.collect()

0

In [5]:
evaluate_list = [evaluate_build(x_test[..., i], 128) for i in range(x_test.shape[-1])]
evaluate_focus_list = []
for evaluate in evaluate_list:
    focus = 1 - (len(evaluate) - 1)/10
    if focus < 0.6:
        focus = 0.6
    evaluate_focus = focus_build(evaluate, focus)
    evaluate_focus_list.append(evaluate_focus)

evaluate_num:   3,focus:0.8000: 3it [00:00, 3013.87it/s]
evaluate_num:   3,focus:0.8000: 3it [00:00, ?it/s]
evaluate_num: 127,focus:0.6000: 127it [00:00, 293.24it/s]
evaluate_num:  99,focus:0.6000: 99it [00:00, 270.60it/s]
evaluate_num:   2,focus:0.9000: 2it [00:00, ?it/s]


In [6]:
model = Sequential([
    hw_layer(evaluate_focus_list, input_shape=(80,5), name='hw_layer'),
    Dense(128, use_bias=False),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Bidirectional(LSTM(128, use_bias=True, return_sequences=True)),
    Dense(32, use_bias=False, activation='selu'),
    Dense(1, name='hw_embedding')
])

In [7]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='mae')
# model.compile(optimizer='adam', loss='mse', metrics='mae')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hw_layer (hw_layer)          (None, 80, 234)           0         
_________________________________________________________________
dense (Dense)                (None, 80, 128)           29952     
_________________________________________________________________
bidirectional (Bidirectional (None, 80, 256)           263168    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 80, 256)           394240    
_________________________________________________________________
dense_1 (Dense)              (None, 80, 32)            8

In [ ]:
callback_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.7,  patience=8, min_lr=1e-7, verbose=True),
    EarlyStopping(monitor='val_loss', patience=32),
    ModelCheckpoint(f'models/{fname}_best.h5', monitor='val_loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir=f'./Log/{fname}')
]

model.fit(x_train, y_train, epochs=1000, batch_size=100,
            validation_data=(x_valid, y_valid), validation_batch_size=1000,
            callbacks=callback_list)

Epoch 1/1000
604/604 [==============================] - 96s 149ms/step - loss: 3.4483 - val_loss: 2.3855
Epoch 2/1000
604/604 [==============================] - 90s 149ms/step - loss: 2.2313 - val_loss: 1.7051
Epoch 3/1000
604/604 [==============================] - 91s 151ms/step - loss: 1.7796 - val_loss: 1.9060
Epoch 4/1000
604/604 [==============================] - 96s 159ms/step - loss: 1.5913 - val_loss: 1.5081
Epoch 5/1000
604/604 [==============================] - 97s 161ms/step - loss: 1.4362 - val_loss: 1.4549
Epoch 6/1000
604/604 [==============================] - 93s 153ms/step - loss: 1.3237 - val_loss: 1.3615
Epoch 7/1000
604/604 [==============================] - 100s 166ms/step - loss: 1.2410 - val_loss: 1.2182
Epoch 8/1000
604/604 [==============================] - 105s 173ms/step - loss: 1.1816 - val_loss: 1.2376
Epoch 9/1000
164/604 [=======>......................] - ETA: 1:02 - loss: 1.1609